In [10]:
import sys
import numpy as np
import pandas as pd
import joblib
import shap  # Import SHAP library
import seaborn as sns  # Import Seaborn for visualization
import matplotlib.pyplot as plt  # Import matplotlib for plotting
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score,make_scorer, classification_report, precision_recall_fscore_support
from sklearn.metrics import classification_report
from scipy import sparse as sp
from pathlib import Path

# Add project root to the Python path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Loading dataset
# Loading dataset

import sys
from pathlib import Path

# Adjust the system path to include the project root directory. 
# It assumes the notebook is inside 'scripts/model_hyperparameter_tuning'.
project_root = Path.cwd().parents[1]
sys.path.insert(0, str(project_root))

# Import the project configuration settings.
import config
from config import RANDOM_SEED
config.set_seed()

# Loading training data
X_test = config.load_data('X_test_classification_WAITTIME_BINARY.csv', 'test')
y_test = config.load_data('y_test_classification_WAITTIME_BINARY.csv', 'test')

# Loading preprocessed data
X_test_preprocessed = config.load_data('X_test_preprocessed_classification_WAITTIME_BINARY.csv', 'processed')

# Load the entire dataset as a DataFrame
X_test_selected_features = config.load_data('X_test_selected_features_WAITTIME_BINARY.csv', 'test').sort_index(axis=1)

In [11]:
# Define the paths for the model and test data
model_name = 'best_waittime_classification_model_tuned.joblib'
# Load the trained model using the utility function from config.py
model = config.load_model(model_name)
print("Model loaded successfully.")


# Predict class labels for the test set
y_pred = model.predict(X_test_selected_features)

# Predict probabilities for the test set
y_pred_proba = model.predict_proba(X_test_selected_features)[:, 1]  # Probabilities for the positive class

# Calculate the evaluation metrics
f1_validation = f1_score(y_test, y_pred, average='weighted')
roc_auc_validation = roc_auc_score(y_test, y_pred_proba)  # Ensure y_test is properly encoded for binary classification
precision_validation = precision_score(y_test, y_pred, average='weighted')
recall_validation = recall_score(y_test, y_pred, average='weighted')

# Print the evaluation metrics
print(f"Validation Metrics:\nF1 (Weighted): {f1_validation:.2f}, ROC AUC: {roc_auc_validation:.2f}, Precision (Weighted): {precision_validation:.2f}, Recall (Weighted): {recall_validation:.2f}")

#Metrics for each class
# Generate a classification report
# Correct usage of classification_report to compare true labels and predicted labels
report = classification_report(y_test, y_pred, output_dict=True)

# Print out the classification report
print(classification_report(y_test, y_pred))

# Access specific metrics from the report dictionary for custom output, if needed
for label, metrics in report.items():
    if label not in ('accuracy', 'macro avg', 'weighted avg'):
        print(f"Class {label} - Precision: {metrics['precision']:.2f}, Recall: {metrics['recall']:.2f}, F1-score: {metrics['f1-score']:.2f}")


Model loaded successfully.
Validation Metrics:
F1 (Weighted): 0.73, ROC AUC: 0.80, Precision (Weighted): 0.74, Recall (Weighted): 0.73
              precision    recall  f1-score   support

         0.0       0.81      0.72      0.76      1777
         1.0       0.62      0.73      0.67      1104

    accuracy                           0.73      2881
   macro avg       0.72      0.73      0.72      2881
weighted avg       0.74      0.73      0.73      2881

Class 0.0 - Precision: 0.81, Recall: 0.72, F1-score: 0.76
Class 1.0 - Precision: 0.62, Recall: 0.73, F1-score: 0.67
